## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [1]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [3]:
label_id = '1USY2RYvnDzH7wXWQF8j6EAtb1YZKg9rD'
download_file(label_id, 'cool_roof_yolo_labels.zip')

Downloading...
From: https://drive.google.com/uc?id=1USY2RYvnDzH7wXWQF8j6EAtb1YZKg9rD
To: /content/cool_roof_yolo_labels.zip
100%|██████████| 63.0k/63.0k [00:00<00:00, 46.7MB/s]


In [4]:
file_id = '1lsa2g2yNu0ZhUp-G8x_agUOJM0luUqc7'
download_file(file_id, 'cool_roof_images.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1lsa2g2yNu0ZhUp-G8x_agUOJM0luUqc7
From (redirected): https://drive.google.com/uc?id=1lsa2g2yNu0ZhUp-G8x_agUOJM0luUqc7&confirm=t&uuid=57aa4135-9189-43a9-a56a-e9295840429c
To: /content/cool_roof_images.zip
100%|██████████| 216M/216M [00:01<00:00, 118MB/s]


In [5]:
file_id2 = '1DK5J2etNZr2yBK29RlsAGwv7A4jfo2E9'
download_file(file_id2, 'cool_roof_images2.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1DK5J2etNZr2yBK29RlsAGwv7A4jfo2E9
From (redirected): https://drive.google.com/uc?id=1DK5J2etNZr2yBK29RlsAGwv7A4jfo2E9&confirm=t&uuid=20ff2da5-6441-43fc-9945-aa3a43f145b3
To: /content/cool_roof_images2.zip
100%|██████████| 71.9M/71.9M [00:01<00:00, 70.8MB/s]


In [6]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [7]:
dataset_extract('cool_roof_images.zip')

Extracting: 100%|██████████| 200/200 [00:01<00:00, 119.87files/s]


In [8]:
dataset_extract('cool_roof_images2.zip')

Extracting: 100%|██████████| 1037/1037 [00:03<00:00, 336.21files/s]


In [9]:
dataset_extract('cool_roof_yolo_labels.zip')

Extracting: 100%|██████████| 204/204 [00:00<00:00, 8951.87files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [10]:
!mkdir yolo
!mkdir yolo/train
!mkdir yolo/valid

In [11]:
!mkdir yolo/train/images
!mkdir yolo/train/labels
!mkdir yolo/valid/images
!mkdir yolo/valid/labels

In [12]:
current_path=os.path.abspath(os.curdir)
IMAGE_PATH=current_path + '/cool_roof_images'
IMAGE_PATH2=current_path + '/cool_roof_images2/images'
LABEL_PATH=current_path + '/cool_roof_yolo_labels/obj_train_data'
LABEL_PATH2=current_path + '/cool_roof_images2/labels'
TRAIN_PATH=current_path + '/yolo/train'
VALID_PATH=current_path + '/yolo/valid'

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [13]:
img_files = os.listdir(IMAGE_PATH)
img_names = [os.path.splitext(file)[0] for file in img_files]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_img, val_img = train_test_split(img_names, test_size=0.2, random_state=2024)
for img in train_img:
    src_path = os.path.join(IMAGE_PATH, img + ".jpg")
    dst_path = os.path.join(TRAIN_PATH + '/images', img + ".jpg")
    src_path2 = os.path.join(LABEL_PATH, img + ".txt")
    dst_path2 = os.path.join(TRAIN_PATH + '/labels', img + ".txt")
    shutil.move(src_path, dst_path)
    shutil.move(src_path2, dst_path2)
for img in val_img:
    src_path = os.path.join(IMAGE_PATH, img + ".jpg")
    dst_path = os.path.join(VALID_PATH + '/images', img + ".jpg")
    src_path2 = os.path.join(LABEL_PATH, img + ".txt")
    dst_path2 = os.path.join(VALID_PATH + '/labels', img + ".txt")
    shutil.move(src_path, dst_path)
    shutil.move(src_path2, dst_path2)

In [16]:
img_files2 = os.listdir(IMAGE_PATH2)
img_names2 = [os.path.splitext(file)[0] for file in img_files2]

In [17]:
train_img2, val_img2 = train_test_split(img_names2, test_size=0.2, random_state=2024)
for img in train_img2:
    src_path = os.path.join(IMAGE_PATH2, img + ".jpg")
    dst_path = os.path.join(TRAIN_PATH + '/images', img + ".jpg")
    src_path2 = os.path.join(LABEL_PATH2, img + ".txt")
    dst_path2 = os.path.join(TRAIN_PATH + '/labels', img + ".txt")

    if os.path.isfile(src_path):
        shutil.move(src_path, dst_path)
        shutil.move(src_path2, dst_path2)
    else:
        src_path = os.path.join(IMAGE_PATH2, img + ".PNG")
        if os.path.isfile(src_path):
            shutil.move(src_path, dst_path)
            shutil.move(src_path2, dst_path2)
for img in val_img2:
    src_path = os.path.join(IMAGE_PATH2, img + ".jpg")
    dst_path = os.path.join(VALID_PATH + '/images', img + ".jpg")
    src_path2 = os.path.join(LABEL_PATH2, img + ".txt")
    dst_path2 = os.path.join(VALID_PATH + '/labels', img + ".txt")

    if os.path.isfile(src_path):
        shutil.move(src_path, dst_path)
        shutil.move(src_path2, dst_path2)
    else:
        src_path = os.path.join(IMAGE_PATH2, img + ".PNG")
        if os.path.isfile(src_path):
            shutil.move(src_path, dst_path)
            shutil.move(src_path2, dst_path2)

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [18]:
import yaml

In [ ]:
# def read_data_file(data_file):
#     data = {}
#     with open(data_file, 'r') as f:
#         for line in f:
#             key, value = line.strip().split('=')
#             data[key.strip()] = value.strip()
#     return data

In [ ]:
# def read_names_file(names_file):
#     with open(names_file, 'r') as f:
#         class_names = f.read().strip().split('\n')
#     return class_names

In [ ]:
# data_file_path = current_path + '/cool_roof_yolo_labels'
# names_file_path = current_path + '/cool_roof_yolo_labels'

In [19]:
yaml_data = {
    'train': TRAIN_PATH,
    'val': VALID_PATH,
    'nc': 2,
    'names': ['cool roof','generic roof']
}
with open('yolo/data.yaml', 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [20]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [21]:
from ultralytics import YOLO, settings

In [22]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [23]:
model = YOLO(model='yolov8m.yaml', task='detect')

In [36]:
model.train(data='/content/yolo/data.yaml',
            epochs=100,
            patience=7,
            pretrained=False,
            verbose=True,
            seed=2024,
            degrees = 0.5,
            translate = 0.1,
            scale = 0.5,
            shear = 0.1,
            hsv_h = 0,
            hsv_s = 0.3,
            hsv_v = 0.3,
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/content/yolo/data.yaml, epochs=100, time=None, patience=7, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train39, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /content/yolo/train/labels.cache... 572 images, 0 backgrounds, 1 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/yolo/train/images/KakaoTalk_20240412_100143804.mp4_20240412100854_0168.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0194]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/yolo/valid/labels.cache... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<?, ?it/s]


Plotting labels to runs/detect/train39/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train39
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     48/100      7.27G      1.327      1.102      1.842        111        640: 100%|██████████| 36/36 [00:20<00:00,  1.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.35it/s]

                   all        144       1212      0.701      0.612       0.66      0.379



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     49/100      7.27G       1.28      1.064      1.818        200        640:  36%|███▌      | 13/36 [00:06<00:11,  1.92it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
     49/100      7.27G       1.28      1.064      1.818        200        640:  36%|███▌      | 13/36 [00:07<00:13,  1.75it/s]


KeyboardInterrupt: 

In [37]:
results = model.predict(source='/content/test3.png',
                        save=True,
                        conf=0.4,
                        iou=0.3,
                        line_width=2)


image 1/1 /content/test3.png: 640x576 2 cool roofs, 20 generic roofs, 38.2ms
Speed: 6.5ms preprocess, 38.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs/detect/train310
